# Waste-Assist Classification¶

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

2021-08-24 11:37:12.032891: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-24 11:37:12.032943: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Preprocessing the Training set

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.3,
                                   horizontal_flip = True,
                                   vertical_flip = True)

# tf.keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False, samplewise_center=False,
#     featurewise_std_normalization=False, samplewise_std_normalization=False,
#     zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
#     height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
#     channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
#     horizontal_flip=False, vertical_flip=False, rescale=None,
#     preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
# )

X_train_gen = train_datagen.flow_from_directory(
        '../raw_data/training',
         target_size = (256, 256),
         batch_size = 32,
         class_mode = 'categorical')

# X_val = test_datagen.flow_from_directory(
#         'data/validation',
#         target_size=(256, 256),
#         batch_size=32,
#         class_mode='categorical')
# flow_from_directory(
#     directory, target_size=(256, 256), color_mode='rgb', classes=None,
#     class_mode='categorical', batch_size=32, shuffle=True, seed=None,
#     save_to_dir=None, save_prefix='', save_format='png',
#     follow_links=False, subset=None, interpolation='nearest'
# )


Found 2678 images belonging to 7 classes.


In [3]:
X_train_gen.class_indices

{'cardboard': 0,
 'compost': 1,
 'glass': 2,
 'metal': 3,
 'paper': 4,
 'plastic': 5,
 'trash': 6}

# Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)
X_test_gen = test_datagen.flow_from_directory('../raw_data/test',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 73 images belonging to 7 classes.


In [5]:
X_test_gen.class_indices

{'cartboard': 0,
 'compost': 1,
 'glass': 2,
 'metal': 3,
 'paper': 4,
 'plastic': 5,
 'trash': 6}

# Building the CNN

In [6]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

def initialize_model():
    '''instanciate and return the CNN architecture'''
    model = Sequential()

    model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256, 256, 3), padding='valid'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3,3), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (2,2), activation='relu', padding='valid'))
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(7, activation='softmax'))

    return model

In [7]:
model = initialize_model()
model.summary()

2021-08-24 11:37:21.522233: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-24 11:37:21.522294: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-24 11:37:21.522320: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-M8A2P9S9): /proc/driver/nvidia/version does not exist
2021-08-24 11:37:21.522688: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 128)       3

# Compiling and Training the CNN

In [8]:
from tensorflow.keras import optimizers

def compile_model(model):
    '''return a compiled model suited for the cifar tasks'''
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', 
                  metrics='accuracy')
    return model

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

baseline = 1/7 # select at random from 7 categories
model = compile_model(model)

es = EarlyStopping(patience=5, verbose=2, baseline = baseline)

In [11]:
history = model.fit(x=X_train_gen,
                    validation_data=X_test_gen,
                    callbacks=[es], 
                    epochs=100, 
                    batch_size=32,
                    use_multiprocessing=True
                   )

2021-08-24 11:37:57.171862: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
84/84 [==============================] - 402s 5s/step - loss: 2.1006 - accuracy: 0.2569 - val_loss: 1.7367 - val_accuracy: 0.2740
Epoch 2/100
30/84 [=========>....................] - ETA: 6:33 - loss: 1.7179 - accuracy: 0.3105

Process Keras_worker_ForkPoolWorker-5:
Traceback (most recent call last):
  File "/home/eugene/.pyenv/versions/3.8.6/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/eugene/.pyenv/versions/3.8.6/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/eugene/.pyenv/versions/3.8.6/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/eugene/.pyenv/versions/3.8.6/lib/python3.8/multiprocessing/queues.py", line 356, in get
    res = self._reader.recv_bytes()
  File "/home/eugene/.pyenv/versions/3.8.6/lib/python3.8/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/eugene/.pyenv/versions/3.8.6/lib/python3.8/multiprocessing/connection.py", line 414, in _recv_bytes
    buf = self._recv(4)
  File "/home/eugene/.pyenv/versions/3.8.6/lib/python3.8/multiprocessing/connection.py", line 379, in _re

KeyboardInterrupt: 

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history)
plt.show()

# Making a single prediction

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/plastic_image.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)
X_train.class_indices

In [ ]:
def prediction():
    for key, value in X_train.class_indices.items():
        if result[0][value] == 1:
            return key
prediction()